In [132]:
from sklearn.datasets import make_moons
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt

from sklearn.svm import SVC, SVR
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics.pairwise import rbf_kernel


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import QuantileTransformer

from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cluster import DBSCAN

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA

from sklearn.model_selection import ShuffleSplit

from scipy.stats import mode
import scipy

In [3]:
moon = datasets.make_moons(n_samples=10000, noise=.4)

In [4]:
X,y = moon

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [150]:
y_test

array([1, 1, 0, ..., 0, 1, 1], dtype=int64)

In [12]:
tree = DecisionTreeClassifier()

In [15]:
np.log2(10**4)

13.287712379549449

In [37]:
tree = DecisionTreeClassifier()

# Define the parameter grid
grid = {
    'max_depth': np.arange(1, 14),  # Adjust the range as needed
    'min_samples_split': np.arange(2, 101),  # Adjust the range as needed
    'min_samples_leaf': np.arange(1, 101),  # Adjust the range as needed
    'max_leaf_nodes': np.arange(1,101)
}

# Create the RandomizedSearchCV object
search = RandomizedSearchCV(tree, grid, n_iter=20, cv=5, scoring='accuracy')

# Fit the RandomizedSearchCV object to your data
search.fit(X_train, y_train)

# Access the best parameters and best score
best_params = search.best_params_
best_score = search.best_score_

In [38]:
best_params

{'min_samples_split': 7,
 'min_samples_leaf': 86,
 'max_leaf_nodes': 39,
 'max_depth': 7}

In [39]:
best_score

0.8594666666666667

In [54]:
pipe = Pipeline([('pca', PCA()),('tree', tree)])

# Define the parameter grid
grid_pca = {
    'tree__max_depth': np.arange(1, 14),  # Adjust the range as needed
    'tree__min_samples_split': np.arange(2, 131),  # Adjust the range as needed
    'tree__min_samples_leaf': np.arange(1, 131),  # Adjust the range as needed
    'tree__max_leaf_nodes': np.arange(1,131)
}


search = RandomizedSearchCV(pipe, grid_pca, n_iter=30, cv=5, scoring='accuracy')

# Fit the RandomizedSearchCV object to your data
search.fit(X_train, y_train)

# Access the best parameters and best score
best_params = search.best_params_
best_score = search.best_score_

In [55]:
best_params

{'tree__min_samples_split': 38,
 'tree__min_samples_leaf': 77,
 'tree__max_leaf_nodes': 61,
 'tree__max_depth': 10}

In [56]:
best_score

0.8590666666666668

In [49]:
y_pred = search.predict(X_test)

In [50]:
accuracy_score(y_test, y_pred)

0.8716

In [79]:
folds = np.zeros((y.shape[0],1000))

In [81]:
folds.shape

(10000, 1000)

# Forest

In [179]:
X_test

array([[ 0.54430169, -0.77053511],
       [ 2.53903653,  0.8230215 ],
       [-1.1971362 ,  0.5747915 ],
       ...,
       [-0.29050296,  0.30535708],
       [ 1.23161968, -0.61971668],
       [ 0.26301573,  0.74828476]])

In [217]:
from sklearn.base import clone

#shuffle
n_trees = 1000

n_instances = 100

rs = ShuffleSplit(n_splits=1000, random_state=42, test_size=len(X_train) - n_instances)



forest = [clone(search.best_estimator_) for i in range(n_trees)]

In [218]:

acc=[]

mini_set = []

for i, (train_index, test_index) in enumerate(rs.split(X_train)):
    X_train_sub = X_train[train_index]
    y_train_sub = y_train[train_index]
    
    mini_set.append((X_train_sub, y_train_sub))



In [219]:
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_set):
    tree.fit(X_mini_train, y_mini_train)


In [220]:
folds = np.empty([n_trees,len(X_test) ], dtype=np.uint8)

In [221]:
folds.shape

(1000, 2500)

In [226]:
for i, tree in enumerate(forest):
    folds[i] = tree.predict(X_test)


y_pred = scipy.stats.mode(folds, axis=0).mode



In [227]:
accuracy_score(y_test, y_pred)

0.502